# Day 1

## Problem 1

At the first Go / No Go poll, every Elf is Go until the Fuel Counter-Upper. They haven't determined the amount of fuel required yet.

Fuel required to launch a given module is based on its mass. Specifically, to find the fuel required for a module, take its mass, divide by three, round down, and subtract 2.

In [4]:
import math
def problem1(mass):
    return math.floor(mass / 3) - 2

In [7]:
# For a mass of 12, divide by 3 and round down to get 4, then subtract 2 to get 2.

problem1(12)

2

In [9]:
# For a mass of 14, dividing by 3 and rounding down still yields 4, so the fuel required is also 2.

problem1(14)

2

In [10]:
# For a mass of 1969, the fuel required is 654

problem1(1969)

654

In [11]:
# For a mass of 100756, the fuel required is 33583.

problem1(100756)

33583

In [31]:
input = """
140403
114038
56226
132100
62000
111395
91372
126850
145044
79273
91088
84429
58971
107626
149678
85268
105251
115850
115947
74982
75008
97761
121022
148319
125244
138640
86968
144443
137218
139558
128776
53593
133805
64245
113120
63085
59209
51671
63956
139163
119501
77432
51040
137313
58973
64708
76505
108041
101124
133219
95907
57933
117791
76209
102960
90848
141969
91297
146254
84585
103447
83172
76648
111340
118543
52957
86004
131965
90898
90909
52217
144674
97058
72387
57962
147792
114025
100193
77582
146708
54283
143979
99582
149890
73229
56045
63240
124091
103324
125187
74027
120344
105333
100939
131454
109570
149398
140535
57379
138385
"""
masses = [int(i) for i in input.split("\n") if i != ""]
fuel_reqs = [problem1(i) for i in masses]
sum(fuel_reqs)

3380731

## Problem 2

Fuel itself requires fuel just like a module - take its mass, divide by three, round down, and subtract 2. However, that fuel also requires fuel, and that fuel requires fuel, and so on. Any mass that would require negative fuel should instead be treated as if it requires zero fuel; the remaining mass, if any, is instead handled by wishing really hard, which has no mass and is outside the scope of this calculation.

So, for each module mass, calculate its fuel and add it to the total. Then, treat the fuel amount you just calculated as the input mass and repeat the process, continuing until a fuel requirement is zero or negative.

In [42]:
def additional_fuel_reqs(fuel_reqs):
    new_reqs = [problem1(i) for i in fuel_reqs if problem1(i) > 0]
    if len(new_reqs) > 0:
        return new_reqs + additional_fuel_reqs(new_reqs)
    else:
        return new_reqs
    
sum(fuel_reqs + additional_fuel_reqs(fuel_reqs))

5068210

# Day 2

## Problem 1

On the way to your gravity assist around the Moon, your ship computer beeps angrily about a "1202 program alarm". On the radio, an Elf is already explaining how to handle the situation: "Don't worry, that's perfectly norma--" The ship computer bursts into flames.

You notify the Elves that the computer's magic smoke seems to have escaped. "That computer ran Intcode programs like the gravity assist program it was working on; surely there are enough spare parts up there to build a new Intcode computer!"

An Intcode program is a list of integers separated by commas (like 1,0,0,3,99). To run one, start by looking at the first integer (called position 0). Here, you will find an opcode - either 1, 2, or 99. The opcode indicates what to do; for example, 99 means that the program is finished and should immediately halt. Encountering an unknown opcode means something went wrong.

Opcode 1 adds together numbers read from two positions and stores the result in a third position. The three integers immediately after the opcode tell you these three positions - the first two indicate the positions from which you should read the input values, and the third indicates the position at which the output should be stored.

For example, if your Intcode computer encounters 1,10,20,30, it should read the values at positions 10 and 20, add those values, and then overwrite the value at position 30 with their sum.

Opcode 2 works exactly like opcode 1, except it multiplies the two inputs instead of adding them. Again, the three integers after the opcode indicate where the inputs and outputs are, not their values.

Once you're done processing an opcode, move to the next one by stepping forward 4 positions.

For example, suppose you have the following program:

```
1,9,10,3,2,3,11,0,99,30,40,50
```

For the purposes of illustration, here is the same program split into multiple lines:

```
1,9,10,3,
2,3,11,0,
99,
30,40,50
```

The first four integers, 1,9,10,3, are at positions 0, 1, 2, and 3. Together, they represent the first opcode (1, addition), the positions of the two inputs (9 and 10), and the position of the output (3). To handle this opcode, you first need to get the values at the input positions: position 9 contains 30, and position 10 contains 40. Add these numbers together to get 70. Then, store this value at the output position; here, the output position (3) is at position 3, so it overwrites itself. Afterward, the program looks like this:

```
1,9,10,70,
2,3,11,0,
99,
30,40,50
```

Step forward 4 positions to reach the next opcode, 2. This opcode works just like the previous, but it multiplies instead of adding. The inputs are at positions 3 and 11; these positions contain 70 and 50 respectively. Multiplying these produces 3500; this is stored at position 0:

```
3500,9,10,70,
2,3,11,0,
99,
30,40,50
```

Stepping forward 4 more positions arrives at opcode 99, halting the program.

Here are the initial and final states of a few more small programs:

- 1,0,0,0,99 becomes 2,0,0,0,99 (1 + 1 = 2).
- 2,3,0,3,99 becomes 2,3,0,6,99 (3 * 2 = 6).
- 2,4,4,5,99,0 becomes 2,4,4,5,99,9801 (99 * 99 = 9801).
- 1,1,1,4,99,5,6,0,99 becomes 30,1,1,4,2,5,6,0,99.

Once you have a working computer, the first step is to restore the gravity assist program (your puzzle input) to the "1202 program alarm" state it had just before the last computer caught fire. To do this, before running the program, replace position 1 with the value 12 and replace position 2 with the value 2. What value is left at position 0 after the program halts?



In [50]:
def day2_step(input, current_index=0):
    opcode = input[current_index]
    if opcode == 1:
        input[input[current_index + 3]] = input[input[current_index + 1]] + input[input[current_index + 2]]
        return day2_step(input, current_index + 4)
    elif opcode == 2:
        input[input[current_index + 3]] = input[input[current_index + 1]] * input[input[current_index + 2]]
        return day2_step(input, current_index + 4)
    elif opcode == 99:
        return input
    else:
        raise ValueError('Invalid opcode')

In [51]:
day2_step([1,9,10,3,2,3,11,0,99,30,40,50])

[3500, 9, 10, 70, 2, 3, 11, 0, 99, 30, 40, 50]

In [45]:
day2_step([1,0,0,0,99])

[2, 0, 0, 0, 99]

In [46]:
day2_step([2,3,0,3,99])

[2, 3, 0, 6, 99]

In [47]:
day2_step([2,4,4,5,99,0])

[2, 4, 4, 5, 99, 9801]

In [48]:
day2_step([1,1,1,4,99,5,6,0,99])

[30, 1, 1, 4, 2, 5, 6, 0, 99]

In [55]:
input = [1,12,2,3,1,1,2,3,1,3,4,3,1,5,0,3,2,6,1,19,1,19,10,23,2,13,23,27,1,5,27,31,2,6,31,35,1,6,35,39,2,39,9,43,1,5,43,47,1,13,47,51,1,10,51,55,2,55,10,59,2,10,59,63,1,9,63,67,2,67,13,71,1,71,6,75,2,6,75,79,1,5,79,83,2,83,9,87,1,6,87,91,2,91,6,95,1,95,6,99,2,99,13,103,1,6,103,107,1,2,107,111,1,111,9,0,99,2,14,0,0]

def run_intcode_command(noun, verb):
    memory = input.copy()
    memory[1] = noun
    memory[2] = verb
    return day2_step(memory)[0]

In [56]:
run_intcode_command(12, 2)

4138687

## Problem 2

"Good, the new computer seems to be working correctly! Keep it nearby during this mission - you'll probably use it again. Real Intcode computers support many more features than your new one, but we'll let you know what they are as you need them."

"However, your current priority should be to complete your gravity assist around the Moon. For this mission to succeed, we should settle on some terminology for the parts you've already built."

Intcode programs are given as a list of integers; these values are used as the initial state for the computer's memory. When you run an Intcode program, make sure to start by initializing memory to the program's values. A position in memory is called an address (for example, the first value in memory is at "address 0").

Opcodes (like 1, 2, or 99) mark the beginning of an instruction. The values used immediately after an opcode, if any, are called the instruction's parameters. For example, in the instruction 1,2,3,4, 1 is the opcode; 2, 3, and 4 are the parameters. The instruction 99 contains only an opcode and has no parameters.

The address of the current instruction is called the instruction pointer; it starts at 0. After an instruction finishes, the instruction pointer increases by the number of values in the instruction; until you add more instructions to the computer, this is always 4 (1 opcode + 3 parameters) for the add and multiply instructions. (The halt instruction would increase the instruction pointer by 1, but it halts the program instead.)

"With terminology out of the way, we're ready to proceed. To complete the gravity assist, you need to determine what pair of inputs produces the output 19690720."

The inputs should still be provided to the program by replacing the values at addresses 1 and 2, just like before. In this program, the value placed in address 1 is called the noun, and the value placed in address 2 is called the verb. Each of the two input values will be between 0 and 99, inclusive.

Once the program has halted, its output is available at address 0, also just like before. Each time you try a pair of inputs, make sure you first reset the computer's memory to the values in the program (your puzzle input) - in other words, don't reuse memory from a previous attempt.

Find the input noun and verb that cause the program to produce the output 19690720. What is 100 * noun + verb? (For example, if noun=12 and verb=2, the answer would be 1202.)

In [57]:
def solve_day2_problem2(x):
    for noun in range(100):
        for verb in range(100):
            try:
                result = run_intcode_command(noun, verb)
            except:
                continue
            else:
                if result == x:
                    return (noun, verb)
                
solve_day2_problem2(19690720)

(66, 35)